In [8]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import tkinter as tk
from tkinter import messagebox

# Fetch and Preprocess Data
Fetch historical stock data and preprocess it for training the LSTM model.

In [9]:
# Fetch historical data for a stock (e.g., Apple)
ticker = "AAPL"
data = yf.download(ticker, start="2015-01-01", end="2023-10-01")

# Extract the 'Close' column
close_prices = data[['Close']].values

# Scale the data using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(close_prices)

# Create sequences of 10 days
sequence_length = 10
X = []
y = []

for i in range(sequence_length, len(scaled_prices)):
    X.append(scaled_prices[i-sequence_length:i, 0])  # Past 10 days' prices
    y.append(scaled_prices[i, 0])  # Next day's price

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# Reshape X to be compatible with LSTM input (samples, timesteps, features)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

print("X shape:", X.shape)
print("y shape:", y.shape)

[*********************100%***********************]  1 of 1 completed

X shape: (2191, 10, 1)
y shape: (2191,)


# Building the LSTM Model
Creating and compile the LSTM model.

In [10]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Display the model summary
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 10, 50)              │          10,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 50)                  │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 25)                  │           1,275 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              26 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 31,901 (124.61 KB)

 Trainable params: 31,901 (124.61 KB)

 Non-trainable params: 0 (0.00 B)

# Train the Model
Splitting the data into training and testing sets and train the model.

In [11]:
# Split the data into training and testing sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.0178 - val_loss: 0.0011
Epoch 2/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 2.3450e-04 - val_loss: 0.0021
Epoch 3/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 2.9866e-04 - val_loss: 0.0011
Epoch 4/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 2.6286e-04 - val_loss: 0.0011
Epoch 5/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 2.2195e-04 - val_loss: 0.0010
Epoch 6/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 2.1198e-04 - val_loss: 0.0010
Epoch 7/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 1.9097e-04 - val_loss: 0.0010
Epoch 8/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 1.9768e-04 - val_loss: 0.0010
Epoch 9/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 2.0501e-04 - val_loss: 0.0011
Epoch 10/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 2.2214e-04 - val_loss: 9.6846e-04
Epoch 11/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 1.9763e-04 - val_loss: 0.0011
Epoch 12/20
55/55 ━

# Creating GUI for User Input
Creating a GUI where the user can input 60 days of stock prices and get a prediction.

In [12]:
from ipywidgets import widgets, interact
import matplotlib.pyplot as plt

# Function to preprocess user input
def preprocess_user_input(user_input, scaler, sequence_length):
    user_input = np.array(user_input).reshape(-1, 1)
    user_input_scaled = scaler.transform(user_input)
    user_input_scaled = np.reshape(user_input_scaled, (1, sequence_length, 1))
    return user_input_scaled

# Function to make a prediction
def predict_next_day_price(model, user_input_scaled, scaler):
    next_day_prediction_scaled = model.predict(user_input_scaled)
    next_day_prediction = scaler.inverse_transform(next_day_prediction_scaled)
    return next_day_prediction[0][0]

# Function to plot original and predicted prices
def plot_prices(original_prices, predicted_price):
    days = list(range(1, len(original_prices) + 2))  # +1 for the predicted day
    plt.plot(days[:-1], original_prices, label="Original Prices", marker='o')
    plt.plot(days[-1], predicted_price, label="Predicted Price", marker='x', markersize=10, color='red')
    plt.xlabel("Day")
    plt.ylabel("Stock Price")
    plt.title("Original vs Predicted Stock Prices")
    plt.legend()
    plt.grid(True)
    plt.show()

# Function to handle user input
def on_submit(*args):
    try:
        user_input = [float(entry.value) for entry in entries]

        if len(user_input) != sequence_length:
            print(f"Error: Please enter exactly {sequence_length} values.")
            return

        user_input_scaled = preprocess_user_input(user_input, scaler, sequence_length)
        predicted_price = predict_next_day_price(model, user_input_scaled, scaler)
        print(f"Predicted Stock Price for the Next Day: {predicted_price:.2f}")
        plot_prices(user_input, predicted_price)

    except ValueError:
        print("Error: Please enter valid numbers in all fields.")

# input fields for 10 days
entries = []
for i in range(sequence_length):
    entry = widgets.FloatText(description=f'Day {i+1}:', layout=widgets.Layout(width='200px'))
    entries.append(entry)

#submit button
submit_button = widgets.Button(description="Submit")
submit_button.on_click(on_submit)

# Display the input fields and submit button
display(widgets.VBox(entries + [submit_button]))